## Load Games Played

This Section loads the details of all the AFL games played from 'https://afltables.com/afl/stats/biglists/bg3.txt'. This is a fixed with file which is transformed and loaded into **GamesPlayed** table.

In [54]:
import sqlite3
data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
table_name='GamesPlayedame'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
drop_query = f"DROP TABLE IF EXISTS {table_name};"
cursor.execute(drop_query)
connection.commit()
connection.close()

In [55]:
import pandas as pd
import numpy as np
import sqlite3

column_names=['A','Date_Played','Round','Team','Team_Score','Opponent','Opponent_Score','Ground']
column_width=[6,18,3,20,17,18,18,20]
data_tppes={'A':int,'Date_Played':str,'Round':str,'Team':str,'Team_Score':str,'Opponent':str,'Opponent_Score':str,'Ground':str}

df=pd.read_fwf('https://afltables.com/afl/stats/biglists/bg3.txt', widths=column_width,names=column_names,skiprows=1)
df.drop('A',axis=1, inplace=True)
df.Round=df.Round.replace(['R1','R2','R3','R4','R5','R6','R7','R8','R9'],
                          ['R01','R02','R03','R04','R05','R06','R07','R08','R09'])
df.Team=df.Team.replace(['Fitzroy','Collingwood','Geelong','South Melbourne','Essendon','St Kilda','Melbourne','Carlton','Richmond','University','Hawthorn','North Melbourne','Footscray','Sydney','West Coast','Brisbane Bears','Adelaide','Fremantle','Western Bulldogs','Brisbane Lions','Port Adelaide','Kangaroos','Gold Coast','GW Sydney'],
                        ['BL','CW','GE','SW','ES','SK','ME','CA','RI','UN','HW','NM','WB','SW','WC','BB','AD','FR','WB','BL','PA','NM','GC','GW'])
df.Opponent=df.Opponent.replace(['Fitzroy','Collingwood','Geelong','South Melbourne','Essendon','St Kilda','Melbourne','Carlton','Richmond','University','Hawthorn','North Melbourne','Footscray','Sydney','West Coast','Brisbane Bears','Adelaide','Fremantle','Western Bulldogs','Brisbane Lions','Port Adelaide','Kangaroos','Gold Coast','GW Sydney'],
                                ['BL','CW','GE','SW','ES','SK','ME','CA','RI','UN','HW','NM','WB','SW','WC','BB','AD','FR','WB','BL','PA','NM','GC','GW'])

df[['Team_Goals','Team_Behind','Team_Points']]=df['Team_Score'].str.split('.',n=3,expand=True)
df[['Opponent_Goals','Opponent_Behind','Opponent_Points']]=df['Opponent_Score'].str.split('.',n=3,expand=True)

# Correct the data types after splitting
df[['Team_Goals', 'Team_Behind', 'Team_Points']] = df[['Team_Goals', 'Team_Behind', 'Team_Points']].astype(int)
df[['Opponent_Goals', 'Opponent_Behind', 'Opponent_Points']] = df[['Opponent_Goals', 'Opponent_Behind', 'Opponent_Points']].astype(int)


# Calculate Team Points and Opponent Points correctly
df['Team_TotalPoints'] = df['Team_Goals'] * 6 + df['Team_Behind']
df['Opponent_TotalPoints'] = df['Opponent_Goals'] * 6 + df['Opponent_Behind']

# Recalculate Team_Result and Opponent_Result using integer comparison
df['Team_Result'] = np.where(df['Team_TotalPoints'] > df['Opponent_TotalPoints'], 1,
                             np.where(df['Team_TotalPoints'] == df['Opponent_TotalPoints'], 9, 0))

df['Opponent_Result'] = np.where(df['Opponent_TotalPoints'] > df['Team_TotalPoints'], 1,
                                 np.where(df['Opponent_TotalPoints'] == df['Team_TotalPoints'], 9, 0))




# df['Team_Result']=np.where(df.Team_Points > df.Opponent_Points,1, np.where(df.Team_Points== df.Opponent_Points,9,0))
# df['Opponent_Result']=np.where(df.Team_Points < df.Opponent_Points,1, np.where(df.Team_Points== df.Opponent_Points,9,0))
df.drop(['Team_Score','Opponent_Score'],axis=1, inplace=True)
df.Date_Played=pd.to_datetime(df.Date_Played,format='%d-%b-%Y')
df['Year']=df.Date_Played.dt.year
# df['Team_Result']=np.where(df.Team_Points > df.Opponent_Points,1, np.where(df.Team_Points== df.Opponent_Points,9,0))
# df['Opponent_Result']=np.where(df.Team_Points < df.Opponent_Points,1, np.where(df.Team_Points== df.Opponent_Points,9,0))
#df.drop(['Team_Points','Opponent_Points'],axis=1, inplace=True)

df=df[['Year']+['Round']+['Team']+['Opponent']+['Date_Played']+['Ground']+['Team_Goals']+['Team_Behind']+['Opponent_Goals']+['Opponent_Behind']+['Team_Result']+['Opponent_Result']]
df=df.astype({'Team_Goals':int,'Team_Behind':int,'Opponent_Goals':int,'Opponent_Behind':int})

# Store the results into GamesPlayed table

data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
connection=sqlite3.connect(data_base)
df.to_sql("GamesPlayed",connection,index=False,if_exists='replace')
# Update 210 grand final draw
query_update=f'''
update GamesPlayed 
set Round = 'GF1'
where year=2010 and round ='GF' and  team_result=9
 '''
connection.execute(query_update)
connection.commit()
connection.close()


In [58]:
# Check that GamesPlayed has been loaded
#
connection=sqlite3.connect(data_base)
query=f''' 
        select 
        *
        from GamesPlayed 
        where 1=1
        --and Year=2024
       -- and  Round='GF'
               
    '''
df=pd.read_sql(query,connection)
connection.close()
df


,Year,Round,Team,Opponent,Date_Played,Ground,Team_Goals,Team_Behind,Opponent_Goals,Opponent_Behind,Team_Result,Opponent_Result
0,1897,R01,BL,CA,1897-05-08 00:00:00,Brunswick St,6,13,2,4,1,0
1,1897,R01,CW,SK,1897-05-08 00:00:00,Victoria Park,5,11,2,4,1,0
2,1897,R01,GE,ES,1897-05-08 00:00:00,Corio Oval,3,6,7,5,0,1
3,1897,R01,SW,ME,1897-05-08 00:00:00,Lake Oval,3,9,6,8,0,1
4,1897,R02,SW,CA,1897-05-15 00:00:00,Lake Oval,6,4,5,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16617,2024,SF,PA,HW,2024-09-13 00:00:00,Adelaide Oval,11,9,11,6,1,0
16618,2024,SF,GW,BL,2024-09-14 00:00:00,Sydney Showground,15,10,15,15,0,1
16619,2024,PF,SW,PA,2024-09-20 00:00:00,S.C.G.,14,11,8,11,1,0
16620,2024,PF,GE,BL,2024-09-21 00:00:00,M.C.G.,12,13,14,11,0,1


In [59]:
#filtered_df =df[df['Year']==2021]
filtered_df=df.query(f'Round=="GF"')
filtered_df

,Year,Round,Team,Opponent,Date_Played,Ground,Team_Goals,Team_Behind,Opponent_Goals,Opponent_Behind,Team_Result,Opponent_Result
131,1898,GF,ES,BL,1898-09-24 00:00:00,Junction Oval,3,5,5,8,0,1
200,1899,GF,BL,SW,1899-09-16 00:00:00,Junction Oval,3,9,3,8,1,0
270,1900,GF,BL,ME,1900-09-22 00:00:00,East Melbourne,3,12,4,10,0,1
341,1901,GF,ES,CW,1901-09-07 00:00:00,Lake Oval,6,7,2,4,1,0
413,1902,GF,CW,ES,1902-09-20 00:00:00,M.C.G.,9,6,3,9,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15775,2020,GF,RI,GE,2020-10-24 00:00:00,Gabba,12,9,7,8,1,0
15982,2021,GF,ME,WB,2021-09-25 00:00:00,Perth Stadium,21,14,10,6,1,0
16189,2022,GF,GE,SW,2022-09-24 00:00:00,M.C.G.,20,13,8,4,1,0
16405,2023,GF,CW,BL,2023-09-30 00:00:00,M.C.G.,12,18,13,8,1,0
